# Getting Started with Subconscious

<a href="https://colab.research.google.com/github/subconscious-systems/subconscious/blob/main/examples/getting_started_notebook/getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

Welcome! This notebook will walk you through **Subconscious** — a long-horizon reasoning engine that chains complex reasoning steps, uses tools, and handles tasks that require extended planning.

**No prior experience required.** We'll go step by step:

1. Install the SDK
2. Set up your API key
3. Run your first agent
4. Give the agent tools (web search, etc.)
5. Get structured output using Pydantic
6. Try different engines

Let's go!

## 1. Install the SDK

Run the cell below to install the Subconscious Python SDK. This is the only dependency you need.

In [ ]:
pip install subconscious-sdk


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 2. Set Your API Key

You'll need a Subconscious API key. If you don't have one yet:

1. Go to [subconscious.dev/platform](https://www.subconscious.dev/platform)
2. Sign up / log in
3. Copy your API key

**Pick the method that matches your environment:**

- **Google Colab** — the cell below will try Colab's built-in Secrets manager first. Add a secret named `SUBCONSCIOUS_API_KEY` via the 🔑 icon in the left sidebar.
- **Local Jupyter** — it will fall back to a `getpass` prompt so your key stays hidden.
- **Anywhere else** — you can paste your key directly into the string below (just don't commit the notebook with it!).

In [8]:
import os

api_key = None

# 1) Colab Secrets (recommended for Colab)
try:
    from google.colab import userdata
    api_key = userdata.get("SUBCONSCIOUS_API_KEY")
    print("Loaded API key from Colab Secrets.")
except Exception:
    pass

# 2) Already set in environment
if not api_key:
    api_key = os.environ.get("SUBCONSCIOUS_API_KEY")
    if api_key:
        print("Loaded API key from environment variable.")

# 3) Interactive prompt (works in Jupyter / terminals)
if not api_key:
    import getpass
    api_key = getpass.getpass("Enter your Subconscious API key: ")

# 4) Last resort — paste it here (don't commit this!)
# api_key = "sk-..."

os.environ["SUBCONSCIOUS_API_KEY"] = api_key
print("API key is set.")

API key is set.


## 3. Initialize the Client

Now we create a `Subconscious` client. This is the object you'll use for every API call.

In [9]:
from subconscious import Subconscious

client = Subconscious(api_key=os.environ["SUBCONSCIOUS_API_KEY"])

print("Client ready!")

Client ready!


## 4. Your First Agent Run (No Tools)

The simplest thing you can do is give the agent a plain instruction and let it reason on its own — no tools, no extra config.

Key parameters:
- **`engine`** — which model to use (we'll start with `"tim-gpt"`)
- **`input.instructions`** — what you want the agent to do
- **`options.await_completion`** — wait for the full answer before returning

In [10]:
run = client.run(
    engine="tim-gpt",
    input={
        "instructions": "Explain what an API is in 3 sentences, as if I'm 10 years old.",
    },
    options={"await_completion": True},
)

print(run.result.answer)

An API is like a menu at a restaurant, showing you what dishes you can order and how to ask for them. When you make a request, the kitchen (which is like another computer or app) prepares what you asked for and gives it back, all without you needing to know how the kitchen works. In the same way, an API helps different programs talk to each other and share information, while keeping things easy and organized.


### Try It Yourself

Change the `instructions` string below to anything you want and run the cell.

In [ ]:
my_question = "What is the difference between machine learning and deep learning?"

run = client.run(
    engine="tim-gpt",
    input={"instructions": my_question},
    options={"await_completion": True},
)

print(run.result.answer)

## 5. Using Platform Tools

Agents become much more powerful when you give them **tools**. Subconscious ships with built-in "platform tools" that require zero setup.

Here are some popular ones:

| Tool | API Name | What it does |
|------|----------|--------------|
| Fast Search | `fast_search` | Quick factual lookups |
| Web Search | `web_search` | Detailed web research |
| News Search | `news_search` | Search news articles |
| Page Reader | `page_reader` | Read a webpage URL |

To give an agent a tool, pass it in the `tools` list. Let's try `fast_search`:

In [ ]:
run = client.run(
    engine="tim-gpt",
    input={
        "instructions": "What is the current population of Tokyo?",
        "tools": [
            {"type": "platform", "id": "fast_search"}
        ],
    },
    options={"await_completion": True},
)

print(run.result.answer)

### Multiple Tools at Once

You can give the agent several tools and it will decide which ones to use. Here we combine `web_search` and `news_search` so the agent can do thorough research:

In [ ]:
run = client.run(
    engine="tim-gpt",
    input={
        "instructions": "Give me a brief summary of the biggest AI news from this week.",
        "tools": [
            {"type": "platform", "id": "web_search"},
            {"type": "platform", "id": "news_search"},
        ],
    },
    options={"await_completion": True},
)

print(run.result.answer)

## 6. Structured Output with Pydantic

Sometimes you don't want a free-text answer — you want **structured data** you can use in code (JSON with specific fields).

Subconscious supports this via [Pydantic](https://docs.pydantic.dev/) models. You define a schema and the agent's answer will match it.

Here's an example that analyzes the sentiment of a piece of text:

In [ ]:
from pydantic import BaseModel


class SentimentAnalysis(BaseModel):
    sentiment: str        # e.g. "positive", "negative", "neutral"
    confidence: float     # 0.0 to 1.0
    keywords: list[str]   # key words that influenced the sentiment


text_to_analyze = "I absolutely loved the new movie! The acting was superb and the storyline kept me on the edge of my seat."

run = client.run(
    engine="tim-gpt",
    input={
        "instructions": f"Analyze the sentiment of this text: '{text_to_analyze}'",
        "answerFormat": SentimentAnalysis,
    },
    options={"await_completion": True},
)

result = run.result.answer
print(f"Sentiment:  {result['sentiment']}")
print(f"Confidence: {result['confidence']}")
print(f"Keywords:   {result['keywords']}")

## 9. Streaming — See Results in Real Time

Instead of waiting for the full run to complete with `client.run()`, you can use **streaming** to receive content as the agent produces it. This is great for long tasks where you want to see thoughts and the answer as they appear.

Call `client.stream()` with the same `engine` and `input` you'd use for `run`. You get an iterator of events:

- **`delta`** — a chunk of content (reasoning and answer are streamed as JSON; we accumulate and print the answer as it arrives)
- **`done`** — the run finished
- **`error`** — something went wrong

Run the cell below to see the answer stream in live.

In [ ]:
import json

# Same client and setup as before — run the API key and "Initialize the Client" cells first!
stream = client.stream(
    engine="tim-gpt",
    input={
        "instructions": "What is 17 * 24? Reply with just the number.",
        "tools": [],  # no tools needed for this
    },
)

full_content = ""
for event in stream:
    if event.type == "delta" and event.content:
        full_content += event.content
        # Print content as it arrives (you'll see raw JSON stream; the final answer is in the "answer" field)
        print(event.content, end="", flush=True)
    elif event.type == "done":
        print("\n\n--- Stream complete ---")
        # Parse and show the final answer from the streamed JSON
        try:
            data = json.loads(full_content)
            if isinstance(data, dict) and "answer" in data:
                print("Answer:", data["answer"])
        except json.JSONDecodeError:
            pass
    elif event.type == "error":
        print(f"\nError: {getattr(event, 'message', 'Unknown error')}")

### Another Structured Example — Extracting Facts

Let's combine tools and structured output. The agent will search the web and return structured data:

In [ ]:
class CompanyInfo(BaseModel):
    name: str
    founded: str
    headquarters: str
    ceo: str
    industry: str


run = client.run(
    engine="tim-gpt",
    input={
        "instructions": "Look up basic facts about SpaceX.",
        "tools": [{"type": "platform", "id": "fast_search"}],
        "answerFormat": CompanyInfo,
    },
    options={"await_completion": True},
)

info = run.result.answer
for field, value in info.items():
    print(f"{field:>15}: {value}")

## 7. Trying Different Engines

Subconscious offers several engines — each optimized for different trade-offs:

| Engine | API Name | Best for |
|--------|----------|----------|
| TIM | `tim` | General-purpose, wide range of tasks |
| TIM-Edge | `tim-edge` | Fast + efficient, great with search tools |
| TIMINI | `timini` | Complex reasoning, backed by Gemini-3 Flash |
| TIM-GPT | `tim-gpt` | Complex reasoning, backed by GPT-4.1 |
| TIM-GPT-Heavy | `tim-gpt-heavy` | Maximum capability, backed by GPT-5.2 |

Just change the `engine` parameter to try a different one:

In [ ]:
engines_to_try = ["tim", "tim-edge", "tim-gpt"]
question = "In one sentence, what causes the Northern Lights?"

for engine in engines_to_try:
    print(f"\n{'=' * 60}")
    print(f"Engine: {engine}")
    print(f"{'=' * 60}")

    run = client.run(
        engine=engine,
        input={
            "instructions": question,
            "tools": [{"type": "platform", "id": "fast_search"}],
        },
        options={"await_completion": True},
    )

    print(run.result.answer)

## 8. Peeking Under the Hood — Reasoning Steps

Every Subconscious run includes a `reasoning` field that shows the agent's step-by-step thought process, including which tools it called and what it found.

This is incredibly useful for understanding *how* the agent arrived at its answer:

In [ ]:
import json

run = client.run(
    engine="tim-gpt",
    input={
        "instructions": "Who won the first ever FIFA World Cup and where was it held?",
        "tools": [{"type": "platform", "id": "fast_search"}],
    },
    options={"await_completion": True},
)

print("ANSWER:")
print(run.result.answer)
print("\nREASONING STEPS:")
print(json.dumps(run.result.reasoning, indent=2))

## What's Next?

You've now covered the core building blocks of Subconscious! Here are some ideas for what to explore next:

- **Custom function tools** — connect the agent to your own HTTP endpoints ([docs](https://docs.subconscious.dev/core-concepts/tools))
- **MCP tools** — plug into Model Context Protocol servers
- **Streaming** — get results in real-time as the agent works
- **More examples** — check out the [examples folder](https://github.com/subconscious-systems/subconscious/tree/main/examples) for full projects

### Useful Links

| Resource | Link |
|----------|------|
| Documentation | [docs.subconscious.dev](https://docs.subconscious.dev) |
| API Reference | [docs.subconscious.dev/api-reference](https://docs.subconscious.dev/api-reference/introduction) |
| Platform (get API key) | [subconscious.dev/platform](https://www.subconscious.dev/platform) |
| GitHub | [github.com/subconscious-systems/subconscious](https://github.com/subconscious-systems/subconscious) |

Happy building!